In [1]:
from importlib.metadata import version

pkgs = [
    "tiktoken",    # 分词器
    "torch",       # 深度学习库
]
for p in pkgs:
    print(f"{p} version: {version(p)}")

tiktoken version: 0.9.0
torch version: 2.6.0+cu126
